In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell, seq2seq
from tqdm import *
from tensorflow.python.ops import control_flow_ops
import sys
import random

from Utils import Utils
import attention_cnn_lstm_lstm as model


    
def test(test_utils, source_len, simplified_len, oseq_len, embedding_size, batch_size, decoder_hidden,
          encoder_hidden, source_nfilters, source_width, lstm_layer, model_dir):
    
    words_size = test_utils.get_words_size()
    
    outputs = model.build_model(words_size=words_size, 
                                embedding_size=embedding_size,
                                source_len=source_len,
                                simplified_len=simplified_len,
                                oseq_len=oseq_len, 
                                decoder_hidden=decoder_hidden,
                                encoder_hidden=encoder_hidden,
                                source_nfilters=source_nfilters,
                                source_width=source_width,
                                lstm_layer=lstm_layer, 
                                batch_size=batch_size, 
                                is_train=False)

    cost=outputs['cost']
    words_prediction=outputs['words_prediction']
    source=outputs['source']
    defendant=outputs['defendant']
    defendant_length=outputs['defendant_length']
    label=outputs['label']
    decoder_inputs=outputs['decoder_inputs']
    loss_weights=outputs['loss_weights']
    keep_prob=outputs['keep_prob']
    sample_rate=outputs['sample_rate']

    #gpu config
    os.environ["CUDA_VISIBLE_DEVICES"] = '2'
    gpu_option = tf.GPUOptions(per_process_gpu_memory_fraction = 0.2)
    session_conf = tf.ConfigProto(allow_soft_placement = True, gpu_options = gpu_option, log_device_placement = False)

    

    tvar = tf.trainable_variables()
    for v in tvar:
        print (v.name)
                        
                                  
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(model_dir)
    
    for path in ckpt.all_model_checkpoint_paths:
        
        sess = tf.Session(config=session_conf)
        saver.restore(sess, path)

        batch = 0
        test_cost = 0
        test_accuracy = 0
        test_bleu = 0
        
        while True:
            
            datas, is_again = test_utils.next_batch()
            if is_again:
                break
                
            batch += 1
            
            
            batch_source = datas['source'] 
            batch_defendant = datas['defendant'] 
            batch_defendant_length = datas['defendant_length']
            batch_ground_truth = datas['ground_truth']
            batch_label = datas['label']
            batch_weights = datas['loss_weights']
            
            feed_dic = {source:batch_source,
                        defendant:batch_defendant,
                        label:batch_label,
                        defendant_length:batch_defendant_length,
                        decoder_inputs:batch_ground_truth,
                        loss_weights:batch_weights,
                        keep_prob:1.,
                        sample_rate:1}
                
            words_, cost_ = sess.run([words_prediction, cost], feed_dict=feed_dic)

            
            sys.stdout.flush()
            accu_, bleu_ = test_utils.i2t(words_, to_print = False)
            
            test_cost += cost_
            test_accuracy += accu_
            test_bleu += bleu_
  

            
        test_cost /= batch
        test_accuracy /= batch
        test_bleu /= batch

        print (path)
        print ("Loss = "+"{:.5f}".format(test_cost) + \
               " Accuracy = "+"{:.5f}".format(test_accuracy) + \
               " BLEU = " + "{:.5f}".format(test_bleu))

        sess.close()
          
        
            
if __name__ == '__main__':
    
    words_path = '/home/xuwenshen/data/big_data/2017_3_13/words'
    test_path = '/home/xuwenshen/data/big_data/2017_3_13/test.h5'
    
    oseq_len = 200
    source_len = 1000
    simplified_len = 150


    batch_size = 50
    decoder_hidden = 700
    encoder_hidden = 312
    source_nfilters = 450
    source_width = 3
    lstm_layer = 1
    embedding_size = 200
    
    
    test_utils = Utils(words_path, test_path, batch_size, nb_samples=10000)

    test(test_utils=test_utils,
          source_len=source_len,
          simplified_len=simplified_len,
          oseq_len=oseq_len, 
          embedding_size=embedding_size,
          batch_size=batch_size,
          decoder_hidden=decoder_hidden,
          encoder_hidden=encoder_hidden,
          source_nfilters=source_nfilters,
          source_width=source_width,
          lstm_layer=lstm_layer,
          model_dir='/home/xuwenshen/2017_3_13/attention_cnn_lstm-lstm/model_v1/')


[None, 998, 1, 450]
[None, 996, 1, 450]
[None, 994, 1, 450]
build model 
embedding:0
conv_source_w_1:0
conv_source_w_2:0
conv_source_w_3:0
conv_source_b_1:0
conv_source_b_2:0
conv_source_b_3:0
conv_source_scale_1:0
conv_source_scale_2:0
conv_source_scale_3:0
conv_source_offset_1:0
conv_source_offset_2:0
conv_source_offset_3:0
generation_w:0
generation_b:0
dynamic_rnn_encoder/BasicLSTMCell/Linear/Matrix:0
dynamic_rnn_encoder/BasicLSTMCell/Linear/Bias:0
rnn_decoder/AttnW_0:0
rnn_decoder/AttnV_0:0
rnn_decoder/Linear/Matrix:0
rnn_decoder/Linear/Bias:0
rnn_decoder/BasicLSTMCell/Linear/Matrix:0
rnn_decoder/BasicLSTMCell/Linear/Bias:0
rnn_decoder/Attention_0/Linear/Matrix:0
rnn_decoder/Attention_0/Linear/Bias:0
rnn_decoder/AttnOutputProjection/Linear/Matrix:0
rnn_decoder/AttnOutputProjection/Linear/Bias:0
/home/xuwenshen/2017_3_13/attention_cnn_lstm-lstm/model_v1/sample_rate-0.5-train_accu-0.000-train_cost-0.695-train_bleu-0.000-valid_accu-0.313-valid_cost-4.354-valid_bleu-0.652-model.ckpt-67

/home/xuwenshen/py3/lib/python3.4/site-packages/nltk/translate/bleu_score.py:472: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
